In [1]:
from sklearn import ensemble
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
from sklearn.linear_model import LinearRegression
from sklearn.neural_network import MLPRegressor
from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit.Chem import DataStructs
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
data = pd.read_csv('SAMPL.csv')

In [3]:
nBits=1024
depth=3

# convertation smiles to morgan fingerprints bit vectors
# (probably not the most pythonic way to do this :)
mols = data['smiles'].apply(Chem.MolFromSmiles)
fps_array = np.zeros((len(mols), nBits))
for i in range(len(mols)):
    mol = mols[i]
    fp = AllChem.GetMorganFingerprintAsBitVect(mol, depth, nBits=nBits)
    fp_array = np.zeros((0,), dtype=np.int8)
    DataStructs.ConvertToNumpyArray(fp, fp_array)
    fps_array[i]= fp_array
fps_array = np.array(fps_array)
fps = pd.DataFrame(fps_array)

In [4]:
# (optional) filter the features to use only the frequent ones
good_indices = []
bound = 15
for i in fps:
    if sum(fps[i]) >= bound:
        good_indices.append(i)
fps_good = fps[good_indices]

In [5]:
# split the data to train and test
X_train, X_test, Y_train, Y_test, Calc_train, Calc_test = train_test_split(fps_good, data['expt'], data['calc'], test_size=0.2, random_state=1337, shuffle=True)

In [6]:
# train gradient boosting regressor
GBR = ensemble.GradientBoostingRegressor(n_estimators=700, max_depth=4, min_samples_split=5)
GBR.fit(X_train, Y_train)

GradientBoostingRegressor(alpha=0.9, criterion='friedman_mse', init=None,
             learning_rate=0.1, loss='ls', max_depth=4, max_features=None,
             max_leaf_nodes=None, min_impurity_decrease=0.0,
             min_impurity_split=None, min_samples_leaf=1,
             min_samples_split=5, min_weight_fraction_leaf=0.0,
             n_estimators=700, n_iter_no_change=None, presort='auto',
             random_state=None, subsample=1.0, tol=0.0001,
             validation_fraction=0.1, verbose=0, warm_start=False)

In [7]:
# train linear regressor
Linear = LinearRegression()
Linear.fit(X_train, Y_train)

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=None,
         normalize=False)

In [8]:
# train multilayer perceptron
MLP = MLPRegressor(hidden_layer_sizes=tuple([nBits]*depth), activation='relu')
MLP.fit(X_train, Y_train)

MLPRegressor(activation='relu', alpha=0.0001, batch_size='auto', beta_1=0.9,
       beta_2=0.999, early_stopping=False, epsilon=1e-08,
       hidden_layer_sizes=(1024, 1024, 1024), learning_rate='constant',
       learning_rate_init=0.001, max_iter=200, momentum=0.9,
       n_iter_no_change=10, nesterovs_momentum=True, power_t=0.5,
       random_state=None, shuffle=True, solver='adam', tol=0.0001,
       validation_fraction=0.1, verbose=False, warm_start=False)

In [9]:
# Calculate errors of each regressor
Y_pred = GBR.predict(X_test)
print('GBR:')
print('MSE:       {}'.format(mean_squared_error(Y_test, Y_pred)))
print('R^2 score: {}'.format(r2_score(Y_test, Y_pred)))
Y_pred = Linear.predict(X_test)
print('Linear:')
print('MSE:       {}'.format(mean_squared_error(Y_test, Y_pred)))
print('R^2 score: {}'.format(r2_score(Y_test, Y_pred)))
Y_pred = MLP.predict(X_test)
print('MLP:')
print('MSE:       {}'.format(mean_squared_error(Y_test, Y_pred)))
print('R^2 score: {}'.format(r2_score(Y_test, Y_pred)))
# Calculate error of method from data (calc column)
print('Calc:')
print('MSE:       {}'.format(mean_squared_error(Y_test, Calc_test)))
print('R^2 score: {}'.format(r2_score(Y_test, Calc_test)))

GBR:
MSE:       3.98812633679
R^2 score: 0.765100377933
Linear:
MSE:       6.44636562047
R^2 score: 0.62031071233
MLP:
MSE:       3.58151090824
R^2 score: 0.789049922764
Calc:
MSE:       3.00768906202
R^2 score: 0.822847882865


In [10]:
# All data
Y_pred = GBR.predict(fps_good)
print('GBR:')
print('MSE:       {}'.format(mean_squared_error(data['expt'], Y_pred)))
print('R^2 score: {}'.format(r2_score(data['expt'], Y_pred)))
Y_pred = Linear.predict(fps_good)
print('Linear:')
print('MSE:       {}'.format(mean_squared_error(data['expt'], Y_pred)))
print('R^2 score: {}'.format(r2_score(data['expt'], Y_pred)))
Y_pred = MLP.predict(fps_good)
print('MLP:')
print('MSE:       {}'.format(mean_squared_error(data['expt'], Y_pred)))
print('R^2 score: {}'.format(r2_score(data['expt'], Y_pred)))
print('Calc:')
print('MSE:       {}'.format(mean_squared_error(data['expt'], data['calc'])))
print('R^2 score: {}'.format(r2_score(data['expt'], data['calc'])))

GBR:
MSE:       0.989074183411
R^2 score: 0.933092262718
Linear:
MSE:       3.36142072408
R^2 score: 0.772610529651
MLP:
MSE:       0.770395275175
R^2 score: 0.947885198563
Calc:
MSE:       2.37627506075
R^2 score: 0.839252514987


In [11]:
# I decided to try different models for baseline: Gradient Boost Regressor, Linear Regressor and Multilayer Perceptron
# All from the sklearn package
# Linear regressor is not doing well, i.e. it shows bad mse and r2 score comparing to gbr and mlp
# GBR and MLP show good results, and MLP dominate GBR a little, so I choose it as a baseline.

# I also noticed that not-shuffled the data when splitting leads to, as I think, overfitting of models so they give scores better than calc
# (i.e. I achieved about 0.87 r2 for GBX against 0.82 by calc, but this configuration is not presented above)